# Hackathon

Some utilities

## Import Utils

In [1]:
!pip install keras
!pip install tables

    100% |################################| 337kB 3.0MB/s ta 0:00:01
    100% |################################| 256kB 4.0MB/s ta 0:00:011
  Running setup.py bdist_wheel for pyyaml ... done
  Stored in directory: /root/.cache/pip/wheels/2c/f7/79/13f3a12cd723892437c0cfbde1230ab4d82947ff7b3839a4fc
Successfully built pyyaml
You are using pip version 9.0.1, however version 9.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |################################| 4.6MB 339kB/s eta 0:00:01
    100% |################################| 399kB 3.9MB/s eta 0:00:01
You are using pip version 9.0.1, however version 9.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [41]:
import keras
from keras.metrics import top_k_categorical_accuracy
import h5py as h5
import numpy as np
import pandas as pd

PATH_DATA = '/notebooks/data/full.h5'
PATH_PREDICT_WITHOUT_GT = '/notebooks/data/pred_students/pred_from_half/pred_eighties_from_half_1_without_gt.h5'
PATH_SUBMIT = '/notebooks/ChallengeHacka/pred_eighties_from_half_1_gr6.csv'
# PATH_PREDICT_WITH_GT = '/pred_teachers/pred_eighties_from_half_1.h5'

In [42]:
BATCH_SIZE = 32
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D, Dropout
import keras.layers.normalization 
from keras.callbacks import Callback
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
def get_idxs(h5_path):
    f = h5.File(h5_path)
    return range(len(f['S2']))

def shuffle_idx(sample_idxs):
    return list(np.random.permutation(sample_idxs))

def split_train_val(sample_idxs, proportion):
    n_samples = len(sample_idxs)
    return sample_idxs[:int((1.-proportion)*n_samples)], sample_idxs[int((1.-proportion)*n_samples):]

def get_batch_count(idxs, batch_size):
    batch_count = int(len(idxs)//batch_size)
    remained_samples = len(idxs)%batch_size
    if remained_samples > 0:
        batch_count += 1

    return batch_count

In [44]:
def generator(h5_path, batch_size, idxs):
    f = h5.File(h5_path, 'r')
    while True : 
        idxs = shuffle_idx(idxs)
        batch_count = get_batch_count(idxs, batch_size)
        for b in range(batch_count):
            batch_idx = idxs[b*batch_size]
            X = f['S2'][batch_idx:batch_idx+batch_size, :,:,:]
            Y = f['TOP_LANDCOVER'][batch_idx:batch_idx+batch_size, :]
            yield np.array(X), keras.utils.np_utils.to_categorical(np.array(Y), 23)

In [45]:
idxs = get_idxs(PATH_DATA)
shuffled_idxs = shuffle_idx(idxs)
train_idxs, val_idxs = split_train_val(shuffled_idxs, 0.2)

print(len(idxs))

18698240


In [46]:
train_gen = generator(PATH_DATA, BATCH_SIZE, train_idxs)
train_batch_count = get_batch_count(train_idxs, BATCH_SIZE)

val_gen = generator(PATH_DATA, BATCH_SIZE, val_idxs)
val_batch_count = get_batch_count(val_idxs, BATCH_SIZE)

In [47]:
print(train_batch_count, val_batch_count)

467456 116864


In [48]:
label = h5.File(PATH_DATA)['TOP_LANDCOVER']
images = h5.File(PATH_DATA)['S2']

In [49]:
def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

def top_5_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=5)

# Instanciation du model

In [12]:
input_shape = (16,16,4)

dropout_rate=0.5

model = Sequential()

model.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.5))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(23, activation='softmax'))

# Fit

In [50]:
# optim = keras.optimizers.Adam(lr=0.001)
optim = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

model.compile(optimizer=optim,
              loss='categorical_crossentropy',
              metrics=['accuracy', top_3_accuracy, top_5_accuracy ])

history = model.fit_generator(train_gen, steps_per_epoch=train_batch_count/20, epochs=1, verbose=1, validation_data=val_gen, nb_val_samples=1000)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=23372.8, epochs=1, verbose=1, validation_steps=1000, validation_data=<generator...)`
  


Epoch 1/1
23373/23372 [==============================] - 2713s 116ms/step - loss: 1.8543 - acc: 0.4113 - top_3_accuracy: 0.7180 - top_5_accuracy: 0.8741 - val_loss: 5.5588 - val_acc: 0.2536 - val_top_3_accuracy: 0.5423 - val_top_5_accuracy: 0.7652


## Prediction routines

In order to submit a result here are some gits

In [51]:
import os 
def prediction_generator(h5_path, batch_size, idxs):
    f = h5.File(h5_path, 'r')

    batch_count = get_batch_count(idxs, batch_size)
    
    for b in range(batch_count):
        batch_idxs = idxs[b*batch_size:(b+1)*batch_size]
        batch_idxs = sorted(batch_idxs)
        X = f['S2'][batch_idxs, :,:,:]
        yield np.array(X)

In [52]:
pred_idx = get_idxs(PATH_PREDICT_WITHOUT_GT)
print(len(pred_idx))
pred_gen = prediction_generator(PATH_PREDICT_WITHOUT_GT, BATCH_SIZE, pred_idx)
prediction = model.predict_generator(pred_gen, steps=get_batch_count(pred_idx, BATCH_SIZE), verbose=1)
print(len(prediction))
# build_h5_pred_file(np.argmax(prediction, axis = 1), PATH_SUBMIT)
panda_prediction = pd.DataFrame(np.argmax(prediction, axis = 1))
panda_prediction.columns = ['TOP_LANDCOVER']
panda_prediction.head()
# panda_prediction.describe()
panda_prediction.to_csv(PATH_SUBMIT, index_label = 'ID')

241700
7554/7554 [==============================] - 25s 3ms/step
241700


In [53]:
unique, counts = np.unique(np.argmax(prediction, axis = 1), return_counts=True)
# panda_prediction.describe()
print(unique)
print(counts)

[ 1  3  4  5 10 12 19 20]
[187315   2795      1   2528     12  48118    141    790]
